In [15]:
import pandas as pd

df = pd.read_csv('data/train.csv')
df = df.drop(columns=['Id'])

In [16]:
df_num = df.select_dtypes(include = ['float64', 'int64'])
df = df_num[['LotArea','MasVnrArea','TotalBsmtSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','SalePrice']]
df = df.dropna()
df.head()

,LotArea,MasVnrArea,TotalBsmtSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,SalePrice
0,8450,196.0,856,1710,548,0,61,0,1,0,2,1,3,1,8,208500
1,9600,0.0,1262,1262,460,298,0,0,0,1,2,0,3,1,6,181500
2,11250,162.0,920,1786,608,0,42,0,1,0,2,1,3,1,6,223500
3,9550,0.0,756,1717,642,0,35,272,1,0,1,0,3,1,7,140000
4,14260,350.0,1145,2198,836,192,84,0,1,0,2,1,4,1,9,250000


In [17]:
# split data set into train and validation
train = df.sample(frac=0.8, random_state=200)
validation = df.drop(train.index)
print(train.shape)
print(validation.shape)

(1162, 16)
(290, 16)


In [18]:
# split the columns into features and target
target = 'SalePrice'
features = df.columns.drop(target)

# put the target in a separate dataframe
train_y = train[target]

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# create the model
model = RandomForestRegressor(n_estimators=100, max_depth=10)

# fit the model
model.fit(train[features], train_y)

# make predictions
validation_predictions = model.predict(validation[features])

# print the mean squared error
print(mean_squared_error(validation[target], validation_predictions))

1586632197.5284817


In [20]:
from sklearn.metrics import r2_score

print(r2_score(validation[target], validation_predictions))

0.7775072084526131
